In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

cpu is available


In [ ]:
trains = torch.utilsls.data.TensorDataset(torch.FloatTensor(x_train).to(device),
                                         torch.FloatTensor(y_train).to(device))

test = torch.utilsls.data.TensorDataset(torch.FloatTensor(x_test).to(device),
                                         torch.FloatTensor(y_test).to(device))

batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=trains, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=batch_size, shuffle=False)

In [5]:
# RNN
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes, num_layers, device):
        super(RNN, self).__init__()
        
        self.device = device # gpu 체크

        self.hidden_size = hidden_size # 히든 사이즈
        
        self.num_layers = num_layers # 레이어 수
        
        self.num_classes = num_classes # 출력 노드 수
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)

    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device) # 초기 히든 스테이트 설정
        
        out, h_out = self.rnn(x, h0)
        h_out = h_out.view(-1, self.hidden_size)
        output = self.fc(h_out)
        return output

In [6]:
# LSTM
class LSTM(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes, num_layers, device):
        super(LSTM, self).__init__()
        
        self.device = device # gpu 체크

        self.hidden_size = hidden_size # 히든 사이즈
        
        self.num_layers = num_layers # 레이어 수
        
        self.input_size = input_size
        
        self.num_classes = num_classes # 출력 노드 수
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size,num_classes) # 출력층

    def forward(self, x):
        
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device) # 초기 히든 스테이트 설정
        
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        return out